In [3]:
from bs4 import BeautifulSoup
import glob
import numpy as np
import pandas as pd
import re
import sys

# Processing LT

In [4]:
df = pd.read_csv("ltn_2023_links_0_processed.csv", index_col=0)
df['date'] = pd.to_datetime(df['date'], format='mixed', errors='coerce')
df = df[~pd.isnull(df['date'])] # Dropped all articles with invalid dates, e.g. https://ec.ltn.com.tw/article/breakingnews/4175363, which has year -0001
df = df.sort_values("date")
df = df.drop_duplicates("title")

/var/folders/g_/c145jppj74d6_h68pzdpnpd80000gn/T/ipykernel_97606/3294988658.py:2: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['date'] = pd.to_datetime(df['date'], format='mixed', errors='coerce')


In [53]:
dfs = []
for i in range(0, 6):
    dfs.append(pd.read_csv(f"ltn_2023_links_{i}_processed.csv", index_col=0))
dfs.append(pd.read_csv("ltn_2023_24_links_processed.csv", index_col=0))
df = pd.concat(dfs)

/var/folders/g_/c145jppj74d6_h68pzdpnpd80000gn/T/ipykernel_97606/2625881571.py:5: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df = pd.concat(dfs)


In [65]:
import datetime
df['date'] = pd.to_datetime(df['date'], format='mixed', errors='coerce')
df = df[~pd.isnull(df['date'])] # Drop all articles with invalid dates, e.g. https://ec.ltn.com.tw/article/breakingnews/4175363, which has year -0001. All of these articles are entertainment section articles.
df = df.sort_values("date")
df = df.drop_duplicates("title")

In [72]:
from datetime import datetime
df = df[pd.to_datetime(df['date'], utc=True) < '2024-01-31 4:00']

In [75]:
df.to_parquet("lt_2023_contents.parquet")

# All together

In [82]:
import pandas as pd
dfs = []
dfs.append(pd.read_parquet("ltn_2020_contents.parquet"))
dfs.append(pd.read_parquet("ltn_2021_contents.parquet"))
dfs.append(pd.read_csv("ltn_2022_links_processed.csv", index_col=0))
dfs.append(pd.read_parquet("ltn_2023_contents.parquet"))

In [83]:
df = pd.concat(dfs).drop_duplicates("title")

In [84]:
df = df[df['video_only'] != True]

In [85]:
df[df['date'].apply(lambda x: type(x) == str)].head(50)

,title,date,text,link,video_only
3,站台力挺林靜儀 潘孟安：有人打在地人招牌「摜籃假燒金」,2022-01-02T19:53:48+08:00,林靜儀沙鹿造勢大會人氣旺。（記者張軒哲攝）〔記者張軒哲／台中報導〕台中市第二選區立委補選投票...,https://news.ltn.com.tw/news/politics/breaking...,NaN
4,批中國防疫矯枉過正 越南要求廣西重開邊境紓解貨車塞車,2022-01-02T19:53:36+08:00,中國堅守防疫清零政策導致大輛貨車卡在中越邊境關卡。（圖擷取自越南新聞社）〔編譯張沛元／綜合報...,https://news.ltn.com.tw/news/world/breakingnew...,NaN
5,官員：Omicron變異株可能使以色列達到群體免疫,2022-01-02T19:50:58+08:00,以色列最高防疫官員今天表示，COVID-19（2019冠狀病毒疾病）Omicron變異株病例...,https://news.ltn.com.tw/news/world/breakingnew...,NaN
9,高雄大樓傍晚陽台冒煙嚇壞住戶 14輛消防車火速到場灌救,2022-01-02T19:43:33+08:00,高雄市左營區文強路某棟11層樓的大樓傍晚發生火警，住戶飽受嚇壞火速疏散，所幸只是陽台雜物燃燒...,https://news.ltn.com.tw/news/society/breakingn...,NaN
10,台北近郊喜見二級珍稀保育動物 藍腹鷴現身四獸山區,2022-01-02T19:41:15+08:00,台北市的近郊地區如四獸山區、南港山區森林，近來在台北市政府大地工程處持續生態調查下，發現藍腹...,https://news.ltn.com.tw/news/life/breakingnews...,NaN
13,老奶奶推壞娃車 清潔隊回收二手娃車改裝贈送,2022-01-02T19:35:41+08:00,中市大安區清潔隊改造二手娃娃車贈送，助老郭奶奶方便推車倒垃圾，汰換掉原本老舊不堪的娃車。（環...,https://news.ltn.com.tw/news/life/breakingnews...,NaN
14,誰買了85萬滷味？ 店家開發票搞烏龍 焦急PO網追回,2022-01-02T19:33:51+08:00,這家位於台北大同區的滷味店，元旦這天烏龍開出超過85萬的發票，焦急PO網協尋顧客。（圖翻攝自...,https://news.ltn.com.tw/news/life/breakingnews...,NaN
18,春節連假北返高峰初三、初四 高公局公布「較好走時段」,2022-01-02T19:12:35+08:00,預估北返尖峰日為2月3日及4日（初三、初四），中午過後將出現長時段壅塞。（資料照，本報合成）...,https://news.ltn.com.tw/news/life/breakingnews...,NaN
19,中國化工大省調電費 尖峰最高漲70％,2022-01-02T19:12:21+08:00,今年元旦開始，江蘇、山東等化工大省宣布調整電價，用電尖峰期漲幅最高達70%。（示意圖，路透社...,https://ec.ltn.com.tw/article/breakingnews/378...,NaN
22,倒數一週！蔡英文站台籲終結政治惡鬥 1/9票投林靜儀,2022-01-02T19:07:04+08:00,蔡英文總統力挺林靜儀。（記者張軒哲攝）〔記者張軒哲／台中報導〕台中市第二選區立委補選投票日倒...,https://news.ltn.com.tw/news/politics/breaking...,NaN


In [86]:
df['source'] = 'lt'
df = df.drop('video_only', axis=1)

In [87]:
date = pd.to_datetime(df['date'], errors='coerce', utc=True)
df['date'] = date 
df = df[~df['date'].isnull()]
df = df.sort_values('date')

In [91]:
df = df[df['date'].dt.year > 2000] # Removing pesky 1970 dates

In [93]:
df.to_parquet("lt_contents.parquet")

/Users/ethanelasky/anaconda3/lib/python3.11/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
